<a href="https://colab.research.google.com/github/navreen-waraich/Dubai-Real-Estate-ROI-Analysis/blob/main/Dubai_Real_Estate_ROI_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The Dubai Revenue Frontier: A 2026 Multi-Channel Property Analysis
**Author:** Navreen Kaur Waraich\
**Tools:** R (Google Colab), Looker Studio\
**Data Sources:** Kaggle Dubai Property Dataset + 2026 Market Benchmarks (Rent, Airbnb, Service Charges)

## Phase 1: Data Integrity Audit
Before performing financial modeling, I am auditing the primary dataset to ensure feasibility.
1. **Schema Mapping:** The primary Kaggle dataset lacks rental income and service charge data. To calculate ROI, I must integrate external market benchmarks.
2. **Key Verification:** I need to extract the exact unique neighborhood names from the primary file to ensure my secondary CSVs (Rent, Service Charges, Airbnb) will join correctly without "Data Mismatch" errors.
3. **Data Quality Check:** Identifying missing values and data types (numeric vs. text) for 'Price' and 'Size' to plan the necessary cleaning steps.

In [4]:
# 1. Load your main dataset
df <- read.csv("properties_data.csv")

# 2. Get a clean list of unique Neighborhoods
# This is the 'Key' for all 3 of your new CSVs
neighborhood_list <- sort(unique(df$neighborhood))
print("--- UNIQUE NEIGHBORHOODS (Use these exact names in your CSVs) ---")
print(neighborhood_list)

# 3. Check 'Quality' values (For your Yield vs Quality Chart)
print("--- UNIQUE QUALITY TIERS ---")
print(unique(df$quality))

# 4. Check Value Ranges (To help you spot 'Outliers' or bad data)
print("--- PRICE & SIZE RANGES ---")
summary(df[c("price", "size_in_sqft")])

# 5. Check for Missing Data (So we know what to clean)
print("--- MISSING VALUES PER COLUMN ---")
colSums(is.na(df))

[1] "--- UNIQUE NEIGHBORHOODS (Use these exact names in your CSVs) ---"
 [1] "Al Barari"                         "Al Barsha"                        
 [3] "Al Furjan"                         "Al Kifaf"                         
 [5] "Al Quoz"                           "Al Sufouh"                        
 [7] "Arjan"                             "Barsha Heights (Tecom)"           
 [9] "Bluewaters"                        "Business Bay"                     
[11] "City Walk"                         "Culture Village"                  
[13] "DAMAC Hills"                       "DIFC"                             
[15] "Discovery Gardens"                 "Downtown Dubai"                   
[17] "Dubai Creek Harbour (The Lagoons)" "Dubai Festival City"              
[19] "Dubai Harbour"                     "Dubai Healthcare City"            
[21] "Dubai Hills Estate"                "Dubai Land"                       
[23] "Dubai Marina"                      "Dubai Production City (IMPZ)"     
[25]

     price           size_in_sqft 
 Min.   :  220000   Min.   : 294  
 1st Qu.:  890000   1st Qu.: 840  
 Median : 1400000   Median :1271  
 Mean   : 2085830   Mean   :1417  
 3rd Qu.: 2200000   3rd Qu.:1703  
 Max.   :35000000   Max.   :9576  

[1] "--- MISSING VALUES PER COLUMN ---"


id        neighborhood            latitude           longitude 
                  0                   0                   0                   0 
              price        size_in_sqft      price_per_sqft      no_of_bedrooms 
                  0                   0                   0                   0 
    no_of_bathrooms             quality           maid_room         unfurnished 
                  0                   0                   0                   0 
            balcony       barbecue_area  built_in_wardrobes          central_ac 
                  0                   0                   0                   0 
childrens_play_area      childrens_pool           concierge     covered_parking 
                  0                   0                   0                   0 
 kitchen_appliances   lobby_in_building        maid_service           networked 
                  0                   0                   0                   0 
       pets_allowed      private_garden         private_gym     private_jacuzzi 
                  0                   0                   0                   0 
       private_pool            security          shared_gym         shared_pool 
                  0                   0                   0                   0 
         shared_spa               study     vastu_compliant    view_of_landmark 
                  0                   0                   0                   0 
      view_of_water      walk_in_closet 
                  0                   0

### **Phase 2.1: External Rent Data Integration**
To calculate ROI, I am integrating actual January 2026 rental benchmarks. I am creating a reference lookup table directly in R.\
The primary dataset contains listing prices but lacks rental income. By joining this with a verified Market Rent table, I can calculate **Gross Yield** and **Net ROI**.


**Data Integrity Steps:**
* **Schema Matching:** Neighborhood names are mapped exactly to the primary dataset's unique keys (verified in Phase 1).
* **Real-World Benchmarks:** Rental values are based on the Jan 2026 Dubai Land Department (DLD) rental index averages.

In [5]:
# Define the neighborhood names and their corresponding 2026 average rents
neighborhoods <- c(
  "Al Barsha", "Al Furjan", "Arjan", "Barsha Heights (Tecom)",
  "Business Bay", "City Walk", "DAMAC Hills", "DIFC",
  "Discovery Gardens", "Downtown Dubai", "Dubai Creek Harbour (The Lagoons)",
  "Dubai Hills Estate", "Dubai Land", "Dubai Marina",
  "Dubai Production City (IMPZ)", "Dubai Silicon Oasis",
  "Dubai South (Dubai World Central)", "Dubai Sports City", "Greens",
  "International City", "Jumeirah Beach Residence", "Jumeirah Lake Towers",
  "Jumeirah Village Circle", "Jumeirah Village Triangle", "Meydan",
  "Motor City", "Palm Jumeirah", "The Views", "Town Square"
)

rents <- c(
  75000, 80000, 65000, 85000, 105000, 160000, 95000, 165000, 55000,
  180000, 115000, 135000, 65000, 125000, 50000, 58000, 48000, 62000,
  105000, 42000, 145000, 90000, 72000, 85000, 110000, 88000, 260000,
  118000, 68000
)

# Create a Data Frame
rent_df <- data.frame(
  neighborhood = neighborhoods,
  avg_yearly_rent = rents
)

# Write to a CSV file in your Colab folder
write.csv(rent_df, "rent_data.csv", row.names = FALSE)

# Verify the file was created
print("rent_data.csv has been created and saved to your directory.")
head(rent_df)

[1] "rent_data.csv has been created and saved to your directory."


,neighborhood,avg_yearly_rent
,<chr>,<dbl>
1,Al Barsha,75000
2,Al Furjan,80000
3,Arjan,65000
4,Barsha Heights (Tecom),85000
5,Business Bay,105000
6,City Walk,160000


### **Phase 2.2: Service Charge Data Integration**
To move from Gross Yield to Net ROI, I am integrating the official DLD Service Charge Index (2026).\
Profitability in Dubai varies significantly based on building maintenance costs. High-end areas like Downtown or Palm Jumeirah have higher fees per square foot, which can "eat" into the higher rents.

**Data Integrity Steps:**
* **Uniformity:** The neighborhood list matches the 'Rent Data' perfectly to ensure every property has a corresponding expense value.
* **Calculation Unit:** These rates are in AED per SqFt (psf), which allows for precise calculation based on the 'size_in_sqft' column in the primary dataset.

In [6]:
# Define the neighborhood names (exact match to Rent CSV)
neighborhoods <- c(
  "Al Barsha", "Al Furjan", "Arjan", "Barsha Heights (Tecom)",
  "Business Bay", "City Walk", "DAMAC Hills", "DIFC",
  "Discovery Gardens", "Downtown Dubai", "Dubai Creek Harbour (The Lagoons)",
  "Dubai Hills Estate", "Dubai Land", "Dubai Marina",
  "Dubai Production City (IMPZ)", "Dubai Silicon Oasis",
  "Dubai South (Dubai World Central)", "Dubai Sports City", "Greens",
  "International City", "Jumeirah Beach Residence", "Jumeirah Lake Towers",
  "Jumeirah Village Circle", "Jumeirah Village Triangle", "Meydan",
  "Motor City", "Palm Jumeirah", "The Views", "Town Square"
)

# Service Charge rates (AED per SqFt) based on 2026 DLD Index
sc_rates <- c(
  14.50, 13.00, 12.00, 15.50,
  14.75, 22.00, 14.00, 24.50,
  10.50, 21.00, 17.50, 18.50,
  11.50, 16.10, 10.00, 9.50,
  9.00, 11.00, 14.50, 7.50,
  18.00, 13.65, 12.50, 12.50,
  15.00, 12.00, 25.00, 15.50,
  11.00
)

# Create a Data Frame
sc_df <- data.frame(
  neighborhood = neighborhoods,
  sc_rate_psf = sc_rates
)

# Write to CSV
write.csv(sc_df, "service_charge_data.csv", row.names = FALSE)

print("service_charge_data.csv created successfully.")
head(sc_df)

[1] "service_charge_data.csv created successfully."


,neighborhood,sc_rate_psf
,<chr>,<dbl>
1,Al Barsha,14.50
2,Al Furjan,13.00
3,Arjan,12.00
4,Barsha Heights (Tecom),15.50
5,Business Bay,14.75
6,City Walk,22.00


### **Phase 2.3: Airbnb Market Benchmarking**
To provide a comprehensive investment outlook, I am integrating short-term rental (STR) performance metrics.\
Investors in Dubai often choose between long-term leasing and short-term holiday home rentals. By including **Average Daily Rate (ADR)** and **Occupancy Rate**, I can model a side-by-side comparison of annual revenue potential for both strategies.

**Data Integrity Steps:**
* **Key Alignment:** Ensured referential integrity by using the primary dataset’s unique neighborhood keys as the Single Source of Truth for all external data mapping.
* **Metric Accuracy:** Occupancy rates (as decimals, e.g., 0.68) and ADR (AED) are based on Jan 2026 market snapshots from major STR data providers.

In [7]:
# Define the neighborhood names (exact match to previous CSVs)
neighborhoods <- c(
  "Al Barsha", "Al Furjan", "Arjan", "Barsha Heights (Tecom)",
  "Business Bay", "City Walk", "DAMAC Hills", "DIFC",
  "Discovery Gardens", "Downtown Dubai", "Dubai Creek Harbour (The Lagoons)",
  "Dubai Hills Estate", "Dubai Land", "Dubai Marina",
  "Dubai Production City (IMPZ)", "Dubai Silicon Oasis",
  "Dubai South (Dubai World Central)", "Dubai Sports City", "Greens",
  "International City", "Jumeirah Beach Residence", "Jumeirah Lake Towers",
  "Jumeirah Village Circle", "Jumeirah Village Triangle", "Meydan",
  "Motor City", "Palm Jumeirah", "The Views", "Town Square"
)

# Average Daily Rates (AED) - 2026 Benchmarks
adr_values <- c(
  450, 480, 380, 520,
  683, 950, 550, 850,
  300, 1107, 750, 650,
  400, 889, 320, 310,
  280, 350, 580, 250,
  950, 530, 450, 460,
  620, 420, 1375, 600,
  350
)

# Occupancy Rates (expressed as decimals for easier calculation later)
occupancy_rates <- c(
  0.70, 0.68, 0.72, 0.71,
  0.67, 0.65, 0.62, 0.66,
  0.78, 0.67, 0.66, 0.65,
  0.64, 0.67, 0.70, 0.74,
  0.70, 0.72, 0.70, 0.80,
  0.75, 0.70, 0.72, 0.70,
  0.64, 0.68, 0.69, 0.70,
  0.68
)

# Create a Data Frame
airbnb_df <- data.frame(
  neighborhood = neighborhoods,
  adr = adr_values,
  occupancy = occupancy_rates
)

# Write to CSV
write.csv(airbnb_df, "airbnb_data.csv", row.names = FALSE)

print("airbnb_data.csv created successfully with 2026 performance benchmarks.")
head(airbnb_df)

[1] "airbnb_data.csv created successfully with 2026 performance benchmarks."


,neighborhood,adr,occupancy
,<chr>,<dbl>,<dbl>
1,Al Barsha,450,0.70
2,Al Furjan,480,0.68
3,Arjan,380,0.72
4,Barsha Heights (Tecom),520,0.71
5,Business Bay,683,0.67
6,City Walk,950,0.65


### **Phase 3: Data Integration & Feature Engineering**
In this phase, I perform a sequential join to merge the primary property listings with the external market benchmarks created in Phase 2.

**Technical Approach:**
1. **Sequential Left Join:** I use the `left_join` function to attach Rent, Service Charge, and Airbnb data to the primary dataset using 'neighborhood' as the common key.
2. **Preserving Granularity:** A left join ensures that no original property listings are lost, even if a specific neighborhood benchmark is not available.
3. **Foundation for ROI:** This consolidated dataframe establishes a unified schema, ensuring consistency across all multi-variable financial models.

In [8]:
# Load the necessary library
library(dplyr)

# 1. Load the primary dataset and the 3 secondary CSVs
main_df <- read.csv("properties_data.csv")
rent_df <- read.csv("rent_data.csv")
sc_df   <- read.csv("service_charge_data.csv")
airbnb_df <- read.csv("airbnb_data.csv")

# 2. Perform the sequential Left Join
# We pipe ( %>% ) the data from one join to the next
final_df <- main_df %>%
  left_join(rent_df, by = "neighborhood") %>%
  left_join(sc_df, by = "neighborhood") %>%
  left_join(airbnb_df, by = "neighborhood")

# 3. Quick Audit of the new dataset
print("--- New Columns Added ---")
print(colnames(final_df))

# 4. Check how many properties successfully matched with our benchmarks
# (Non-NA values in the new columns)
matched_count <- sum(!is.na(final_df$avg_yearly_rent))
total_count <- nrow(final_df)

print(paste("Successfully matched", matched_count, "out of", total_count, "properties."))

# 5. Save this intermediate 'Merged' file
write.csv(final_df, "merged_properties_data.csv", row.names = FALSE)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




[1] "--- New Columns Added ---"
 [1] "id"                  "neighborhood"        "latitude"           
 [4] "longitude"           "price"               "size_in_sqft"       
 [7] "price_per_sqft"      "no_of_bedrooms"      "no_of_bathrooms"    
[10] "quality"             "maid_room"           "unfurnished"        
[13] "balcony"             "barbecue_area"       "built_in_wardrobes" 
[16] "central_ac"          "childrens_play_area" "childrens_pool"     
[19] "concierge"           "covered_parking"     "kitchen_appliances" 
[22] "lobby_in_building"   "maid_service"        "networked"          
[25] "pets_allowed"        "private_garden"      "private_gym"        
[28] "private_jacuzzi"     "private_pool"        "security"           
[31] "shared_gym"          "shared_pool"         "shared_spa"         
[34] "study"               "vastu_compliant"     "view_of_landmark"   
[37] "view_of_water"       "walk_in_closet"      "avg_yearly_rent"    
[40] "sc_rate_psf"         "adr"             

### **Phase 4.1: Quality-Based Data Imputation**
After the master join, approximately 12% of the properties lacked external market benchmarks due to neighborhood name variances.

**The Solution:**
Rather than discarding these data points, I am performing **imputation**. I calculate the mean Rent, Service Charges, and Airbnb performance for each 'Quality' tier (Ultra, High, Medium, Low) and apply these averages to the missing records. This preserves the dataset's size while maintaining statistical reliability for the geographic heatmap.

In [9]:
# 1. Create a reference table of averages based on 'Quality'
imputation_lookup <- final_df %>%
  group_by(quality) %>%
  summarise(
    avg_rent_imp = mean(avg_yearly_rent, na.rm = TRUE),
    sc_rate_imp = mean(sc_rate_psf, na.rm = TRUE),
    adr_imp = mean(adr, na.rm = TRUE),
    occ_imp = mean(occupancy, na.rm = TRUE)
  )

# 2. Map these averages back to the main dataframe only where values are NA
clean_df <- final_df %>%
  left_join(imputation_lookup, by = "quality") %>%
  mutate(
    avg_yearly_rent = ifelse(is.na(avg_yearly_rent), avg_rent_imp, avg_yearly_rent),
    sc_rate_psf = ifelse(is.na(sc_rate_psf), sc_rate_imp, sc_rate_psf),
    adr = ifelse(is.na(adr), adr_imp, adr),
    occupancy = ifelse(is.na(occupancy), occ_imp, occupancy)
  ) %>%
  select(-ends_with("_imp")) # Clean up temporary columns

# 3. Verification: Check if any NAs remain in our key financial columns
print("Missing values after imputation:")
colSums(is.na(clean_df[c("avg_yearly_rent", "sc_rate_psf", "adr", "occupancy")]))

[1] "Missing values after imputation:"


avg_yearly_rent     sc_rate_psf             adr       occupancy 
              0               0               0               0

### **Phase 4.2: Financial Feature Engineering**
With a complete dataset, I am now engineering the performance metrics required for a dual-path investment analysis.

**Metrics Calculated:**
1. **Operating Expenses:** Converting PSF (Per Square Foot) service charges into an absolute annual cost based on unit size.
2. **Path A (Long-Term Lease):** Calculating Net Profit after deducting a standard 5% property management fee and annual service charges.
3. **Path B (Short-Term/Airbnb):** Modeling potential revenue based on ADR and Occupancy, deducting a 20% specialized holiday-home management fee and service charges.
4. **Yield & ROI:** Calculating the percentage return on the initial purchase price for both investment strategies.

In [10]:
# Calculate all financial metrics using 'mutate'
clean_df <- clean_df %>%
  mutate(
    # 1. Total Annual Operating Cost (Service Charges)
    annual_service_charge = size_in_sqft * sc_rate_psf,

    # 2. Long-Term Strategy Calculations
    # Formula: Rent - Service Charges - 5% Management Fee
    net_profit_long = avg_yearly_rent - annual_service_charge - (0.05 * avg_yearly_rent),
    roi_long_pct = (net_profit_long / price) * 100,

    # 3. Airbnb Strategy Calculations
    # Formula: (Daily Rate * 365 * Occ) - Service Charges - 20% Management Fee
    annual_airbnb_rev = adr * 365 * occupancy,
    net_profit_airbnb = annual_airbnb_rev - annual_service_charge - (0.20 * annual_airbnb_rev),
    roi_airbnb_pct = (net_profit_airbnb / price) * 100
  )

# Quick check on the results
print("Summary of New Financial Metrics:")
summary(clean_df[c("roi_long_pct", "roi_airbnb_pct", "annual_service_charge")])

[1] "Summary of New Financial Metrics:"


  roi_long_pct     roi_airbnb_pct     annual_service_charge
 Min.   :-0.1143   Min.   : 0.05419   Min.   :  2907       
 1st Qu.: 4.6330   1st Qu.: 6.41412   1st Qu.: 13000       
 Median : 7.1958   Median : 9.60873   Median : 20880       
 Mean   : 8.0563   Mean   :10.70134   Mean   : 25434       
 3rd Qu.:10.4482   3rd Qu.:13.56165   3rd Qu.: 30654       
 Max.   :48.2194   Max.   :62.18741   Max.   :239400       

### **Phase 4.3: Statistical Scrubbing**
To ensure the integrity of the visualization layer in Looker Studio, I am applying a statistical filter to remove extreme outliers. This prevents data entry errors or unique "distress sales" from skewing the market averages on the geographic heatmap.

**Filter Parameters:**
* **ROI Floor:** -5% (to allow for properties with very high service charges).
* **ROI Ceiling:** 25% (to remove statistically improbable returns often caused by listing errors).

In [11]:
# 1. Apply the statistical filters
final_export_df <- clean_df %>%
  filter(roi_long_pct > -5 & roi_long_pct < 25) %>%
  filter(roi_airbnb_pct > 0 & roi_airbnb_pct < 35)

# 2. Compare the data before and after the scrub
print(paste("Original rows:", nrow(clean_df)))
print(paste("Rows after scrub:", nrow(final_export_df)))
print(paste("Total outliers removed:", nrow(clean_df) - nrow(final_export_df)))

# 3. Final verification of the ROI range
summary(final_export_df[c("roi_long_pct", "roi_airbnb_pct")])

[1] "Original rows: 1905"
[1] "Rows after scrub: 1891"
[1] "Total outliers removed: 14"


  roi_long_pct     roi_airbnb_pct    
 Min.   :-0.1143   Min.   : 0.05419  
 1st Qu.: 4.6174   1st Qu.: 6.37908  
 Median : 7.1544   Median : 9.53827  
 Mean   : 7.8473   Mean   :10.45877  
 3rd Qu.:10.2524   3rd Qu.:13.53713  
 Max.   :24.3347   Max.   :32.33437  

### **Phase 5: Final Data Export**

In [12]:
# Export the finalized 'Golden Dataset' to CSV
write.csv(final_export_df, "dubai_investment_analysis.csv", row.names = FALSE)

print("SUCCESS: File generated.")

[1] "SUCCESS: File generated."
